In [ ]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

In [ ]:
from pathlib import Path

import soccer_twos

from src.agents.a2c_agent import A2CAgent
from src.agents.ddpg_agent import DDPGAgent
from src.agents.ddqn_agent import DQNAgent
from src.agents.maddpg_agent import MADDPGAgents
from src.agents.ppo_agent import PPOAgent
from src.config import CHECKPOINT_DIR, N_GAMES
from src.utils import shape_rewards

multiplayer = False
# CHECKPOINT_DIR = "notebooks/data/kaggle_single_out_1000/checkpoints"

In [ ]:
import random


def evaluate_agent(agent, n_games=N_GAMES):
    env = soccer_twos.make(render=True, time_scale=1, quality_level=5, worker_id=3)

    for i in range(n_games):
        obs = env.reset()
        done = False
        scores = {}
        i = 0
        while not done:
            i += 1
            actions = {}
            for j in range(4):
                actions[j] = [0, 0, 0]
                if j < agent.num_agents:
                    if agent.__class__.__name__.lower().replace("agent", "") == "ppo":
                        actions[j], _ = agent.act(obs[j])
                    else:
                        actions[j] = agent.act(obs[j])
                    print(
                        "evaluating agent",
                        j,
                        "in time step",
                        i,
                        "with action",
                        actions[j],
                        "and observation",
                        obs[j][:8],
                    )

            next_obs, reward, done, info = env.step(actions)
            done = done["__all__"]
            shape_rewards(info, 0)
            obs = next_obs

    env.close()

In [ ]:
import glob

multiplayer = True
# Load the best agent (example: checkpoint at episode 90)
best_agent = A2CAgent(336, 3)

agent_name = best_agent.__class__.__name__.lower().replace("agent", "")
pattern = f"best_model_{agent_name}"

matching_files = glob.glob(str(parent_dir / Path(CHECKPOINT_DIR) / f"{pattern}*"))
filename = matching_files[0] if matching_files else None


print(matching_files)
best_agent.load(parent_dir / Path(CHECKPOINT_DIR) / str(matching_files[0]))

# Evaluate the trained agent
evaluate_agent(best_agent, n_games=5)